# Application of Natural Selection in Solving Optimization Problems


### Traveling Salesman Problem:

The Traveling Salesman Problem is a famous combinatorial optimization problem found in theoretical computer science. It's classfied as an _NP-hard_ problem, which for brevity's sake is a problem for which there is __no known polynomial algorithm__, so the time to find a solution grows _exponentially_ with problem size.

The premise of the TSP problem is that we have a set of cities (examplified as points on a Cartesian plane) that need to be traversed. The question asks what the __shortest distance to traverse all of them and return to the original point__ would be.

<img src="demo/png/Figure_1.png" width="800" height="1600" style="float: center;"></img>

When brute-forced (also known as a lexicographic order), i.e. when the distance of __every possible permutation__ is calculated to find the shortest path, the time-space complexity of the algorithm is $O(n!)$, where $n$ indicates the number of points on the cartesian plane. To put this into perspective:

- $n=7$ will need `5,040` operations to compute
- $n=14$ will need `87,178,291,200` operations to compute


### Genetic Algorithms:

Belonging to the general class of Evolutionary Algorithms, a Genetic Algorithm is a process inspired by natural selection and its operators such as __mutation__, __crossover (breeding)__, and __selection__.

It can help compute optimal solutions for complex optimization problems in fractions of compute time when compared to other solutions. However, we aren't guaranteed to reach or know that we have reached the most optimal solution, since this, much like evolution itself, is a __heuristic__ process.


### Symbiosis of non-determininistic solutions in deterministic enviornments

Computers are inherently __deterministic__, which is in part why the problems that seem trivial to humans, like the Traveling Salesman Problem are some of the hardest for machines to solve. However, we can mitigate these issues by introducing Genetic Algorithms into play, which take basis in quite non-determined environments, such as our planet. This is a great symbiosis that allows us to leverage both the speed and efficiency of computer systems, but also the elegance of concepts such as Darwin's theory of evolution.

![](demo/png/TSP.png)


### Implementation of a GA to solve the TSP

There are 3 major mechanisms put in place that iteratively simulate evolution, which are (in that order): __Selection__, __Mutation__ and __Crossover__ (also known as Breeding). The iterable, is the Population, which is generated before iteration begins and is modified across Generations.

#### Backbone:
For this project, I've defined the "cities" that the salesman must traverse as a list of points on a plane. For example:

```python
x = np.array([[20, 40], [40, 20], [60, 20], [80, 40], [80, 60], [60, 80], [40, 80], [20, 60]])
```

The above example code produces an octagon produced by 8 points, with side length $20$ and diagonal length $\sqrt{2(20^2)}$, with points plotted as a scatter graph.

#### Initial Population:
Members of the Population however, are initialized as random __shuffles__, or __permutations__ of the cities array.

![](demo/png/Population_Member.png)

```python
np.random.shuffle(x)

> x
[[40, 20], [40, 80], [20, 40], [60, 20], [20, 60], [80, 40], [80, 60], [60, 80]]
```

The permutations are repeated to produce a list of Members that together comprise the Population. 

#### Fitness Scoring:
Members of the randomly generated Population are then measured for their fitness by calculating the __euclidean distance__ between each successive point (including the distance between the last and first points) for each Member. The fitness scores are attached to the respective Members.

#### Selection:
Includes two configurable parameters:

- __survival size__ - default set to $0.5$ - _represents the fraction of the population that must be "saved"_
- __weight__ - default set to $e$ - _weighting used to affect the bias in choosing who gets to survive_

In short, the population is first __ordered by their fitness ranking__, from the shortest distance to the longest distance. Each Member's survival chance is then affected by their position within the Population, as well as the weight multiplier used. The higher the rank of the Member, the higher their chance of survival, and vice versa. The higher the multiplier (weight parameter), the larger the advantage is for the fittest Members (and lower for the least fit).

#### Mutation:
Includes a single configurable parameter:

- __chance__ - default set to $0.35$ - _the chance that mutation occurs for a Member in the Population_

If a Member is randomly selected for mutation, then two random points found in the Member are __swapped__. This can be compared to base substitution in DNA. The reason for a relatively high mutation chance set as default is because of the fact that swapping is not always a major alteration to the Member, especially since only a single swap is done.

![](demo/png/Mutation.png)

#### Crossover (Breeding):
Requires two Members (parents) to produce a single Member (child)

- __parent_a__ - parent \#1
- __parent_b__ - parent \#2

This is one of the functions that I've spent most time on due to its complexity. An example of standard crossover may be to simply combine the first half of parent `a` and the second half of parent `b`; however, standard crossover functions do not work for the Traveling Salesman Problem, since they may introduce duplicate points and completely ignore others. Therefore, I've developed my own crossover algorithm, that works as follows:

1. a random index __i__ is selected from the array. 

Let's assume that for an array of length 4 , index of 2 is selected.
2.  all points in __parent_a__ until index __i__ are copied to the child.

Therefore, the first half of the points the child receives directly from the 1st parent.

3. The compliment union of points that both __parent_b__ and the __child__ share are derived and added to the child in the same order they appear in __parent_b__.

In layman's terms, the second half of the child is populated by the remaining points in the second parent in the exact same order that they appear in.

![](demo/png/Crossover.png)

```python
a = np.array([[0, 1], [2, 3], [4, 5], [6, 7]])

b = np.array([[2, 3], [6, 7], [0, 1], [4, 5]])

> c
[[0, 1], [2, 3], [6, 7], [4, 5]]
```

Crossover is done with randomly chosen Members that have survived the selection, until the Population is restored back to 100%

#### Iteration and Convergence
All of the above steps are iterated, so with relatively little time, very optimal and adapted solutions are found. We consider the solutions to be found when convergence occurs. However, convergence not always occurs at the global minima, but can also happen at a local minima. To avoid this, careful measures for mutation rates, population size, as well as the weight factor in the selection process must be selected.

For example, if mutation rates are too high, the global minima might be skipped altogether, and if it's too low then convergence is reached too early.

In [ ]:
%matplotlib inline
import numpy as np
from genetic import Points, Route, Population


# initialize our grid size and city (randomly generated points)
city = Points(grid=(100, 100), size=15)

# specify for the Population to be created by shuffling the city points
routes = Population(city.points)

# selection, crossover, mutate REPEAT
for i in range(2000):
    routes.selection()
    routes.crossover()
    routes.mutate(chance=0.4)
    # if plotting from CLI, use animate=True instead
    routes.plot(i, jupyter=True)